In [10]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

pandas for data handling.
torch for PyTorch functionalities.
DataLoader for batching and shuffling data during training.
AutoTokenizer and AutoModelForSequenceClassification from transformers for using ClinicalBERT.
AdamW optimizer from PyTorch.
train_test_split for splitting data into train and test sets.
accuracy_score and classification_report from sklearn.metrics for evaluating the model.

In [2]:
# Load the dataset
df = pd.read_csv("cleaned_data.csv")

# Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df['transcription'], df['medical_specialty'], test_size=0.2, random_state=42)


Load the CSV file containing medical transcriptions and their corresponding medical specialties.
Split the data into training and testing sets, with 80% for training and 20% for testing.

In [3]:
# Load the ClinicalBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")

# Tokenize the texts
train_encodings = tokenizer(train_texts.tolist(), padding=True)
test_encodings = tokenizer(test_texts.tolist(), padding=True)

# Convert labels to numerical values
label2id = {label: idx for idx, label in enumerate(df['medical_specialty'].unique())}
train_labels = [label2id[label] for label in train_labels]
test_labels = [label2id[label] for label in test_labels]


Initialize the ClinicalBERT tokenizer and tokenize the text data for both training and testing sets.
Encode the labels as numerical values using a dictionary mapping.

In [4]:
class MedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MedicalDataset(train_encodings, train_labels)
test_dataset = MedicalDataset(test_encodings, test_labels)


Define a custom PyTorch dataset (MedicalDataset) to work with DataLoader.
Implement __init__, __getitem__, and __len__ methods as per PyTorch dataset requirements.
Create instances of the dataset for both training and testing data.

In [15]:
# Initialize the model
model = AutoModel.from_pretrained("medicalai/ClinicalBERT", num_labels=len(label2id))

# Define training parameters
optimizer = AdamW(model.parameters(), lr=5e-5)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


Some weights of the model checkpoint at medicalai/ClinicalBERT were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialize the ClinicalBERT model for sequence classification with the specified number of labels.
Define the optimizer (AdamW) with a learning rate of 5e-5 and set up the training data loader.

In [6]:
# Training loop
model.train()
for batch in train_loader:
    optimizer.zero_grad()
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['labels']
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()


KeyboardInterrupt: 

Set the model in training mode.
Iterate through batches in the training data loader, perform forward and backward passes, calculate loss, and update model parameters.

In [ ]:
# Evaluation
model.eval()
predictions = []
with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=32):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=1).tolist())
